# THE VACUUM WORLD

## ...

---
<br>

**2. Apply your chosen algorithm to compute an optimal sequence of actions for a 3x3 world whose initial state has dirt in the three top squares and the agent in the center.**

In [5]:
import sys
sys.path.append('../..')
import numpy as np

from search import *
from agents import *

In [6]:
class P(Problem):
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        action_list = ['suck', 'left', 'right', 'up', 'down']
        location = state[1]
        dim = np.sqrt(len(state[0])).astype(int)
        state = np.array(state[0]).reshape(dim,dim)
        
        if location[0] == 1: action_list.remove('up')
        if location[0] == state.shape[0]: action_list.remove('down')
        if location[1] == 1: action_list.remove('left')
        if location[1] == state.shape[1]: action_list.remove('right')
        
        loc_row, loc_col = location[0], location[1]
        if state[loc_row - 1][loc_col - 1] == 0: action_list.remove('suck')
        
        return action_list 
    
    def result(self, state, action):
        location = state[1]
        dim = np.sqrt(len(state[0])).astype(int)
        state = np.array(state[0]).reshape(dim,dim)
        loc_row, loc_col = location[0], location[1]

        ## Note: up means increading the array rows
        if action == 'suck': state[loc_row - 1][loc_col - 1] = 0
        if action == 'up': loc_row -= 1
        if action == 'down': loc_row += 1
        if action == 'left': loc_col -= 1
        if action == 'right': loc_col += 1
        
        
        #if state[loc_row - 1][loc_col - 1] == 1:
        #    state[loc_row - 1][loc_col - 1] = 0
            
        state = state.flatten()
        location = (loc_row, loc_col)
        
        return (tuple(state), location)

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            dim = np.sqrt(len(state[0])).astype(int)
            return np.array_equal(np.array(state[0]).reshape(dim,dim), np.array(self.goal).reshape(dim,dim))

In [7]:
def print_solution(solution):
    node, actions_executed = solution, []

    while node.parent:
        actions_executed.append((node, node.action))
        node = node.parent

    for n in actions_executed[::-1]:
        print('    > Reached Node {} with action {}'.format(n[0], n[1]))

def get_seq(solution):
    node, actions_executed = solution, []
    seq = []

    while node.parent:
        seq.insert(0, node.action)
        node = node.parent

    return seq 

In [8]:
world = ((1, 1, 1, 0, 0, 0, 0, 0, 0),(2,2))
goal  = (0,0,0,0,0,0,0,0,0)

#world = (('dirty','dirty','dirty','clean','clean','clean','clean','clean','clean'),(2,2))
#goal  = ('clean','clean','clean','clean','clean','clean','clean','clean','clean')

problem = P(initial=world, goal=goal)
#solution = breadth_first_tree_search(problem)
#solution = depth_first_tree_search(problem)
#solution = depth_first_graph_search(problem)
#solution = breadth_first_graph_search(problem)
#solution = uniform_cost_search(problem)
#solution = depth_limited_search(problem, limit=50)
#solution = iterative_deepening_search(problem)
solution = best_first_graph_search(problem, lambda node: node.path_cost)

print("Solution state:  ", solution)
print("Solution cost :  ", solution.path_cost)
print("Solution nodes:  ", solution._ids)
print("Solution path :  ")
print_solution(solution)

Solution state:   <Node ((0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1))>
Solution cost :   7


AttributeError: 'Node' object has no attribute '_ids'

<br>

**TODO:**
- List search performances here.
- Nutshell: BFS Graph provides smallest space and optimal solution

---
<br>

**3. Construct a search agent for the vacuum world, and evaluate its performance in a set of 3x3 worlds with probability 0.2 of dirt in each square. Include the search cost as well as path cost in the performance measure, using a reasonable exchange rate.**


In [10]:
class VacuumEnv(Environment):

    """This environment has NxN locations. Each can be Dirty (1)
    or Clean (0). The agent perceives its location and the location's
    status."""

    def __init__(self, dim, p_dirt=0.2):
        super().__init__()
        self.status = np.array([1 if random.uniform(0, 1) < p_dirt else 0 for i in range(0, dim*dim)]).reshape(dim, dim)

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        loc_row, loc_col = agent.location[0], agent.location[1]
        return (agent.program.location, self.status[loc_row - 1][loc_col - 1])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        
        loc_row, loc_col = agent.program.location[0], agent.program.location[1]
        
        print('  > {} in [{},{}]'.format(action, loc_row, loc_col))
        if action == 'suck': 
            self.status[loc_row - 1][loc_col - 1] = 0
            agent.performance += 10
        elif action == 'up': loc_row -= 1
        elif action == 'down': loc_row += 1
        elif action == 'left': loc_col -= 1
        elif action == 'right': loc_col += 1
            
        if action in ['up', 'down', 'left', 'right']: 
            agent.performance -= 1
            
        agent.program.location = (loc_row, loc_col)

    def default_location(self, thing):
        """Agents start in either location at random."""
        #return random.choice([loc_A, loc_B])
        return (1,1)

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        #return not any(agent.is_alive() for agent in self.agents)
        if self.status.sum() == 0: print('Done')
        return self.status.sum() == 0

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    # actions will only get the BEST action according to the search algorithm solution
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

            
class SimpleProblemSolvingAgentProgram:
    """Abstract framework for a problem-solving agent. [Figure 3.1]"""

    def __init__(self, initial_state=None, location=None):
        """State is an abstract representation of the state
        of the world, and seq is the list of actions required
        to get to a particular state from the initial state(root)."""
        self.state = initial_state
        self.location = location
        self.performance = 10
        self.solution = None
        self.seq = []

    def __call__(self, percept):
        """[Figure 3.1] Formulate a goal and problem, then
        search for a sequence of actions to solve it."""
        #self.state = self.update_state(self.state, percept)
        if not self.seq:
            goal = self.formulate_goal(self.state.shape[0])
            problem = self.formulate_problem(self.state, goal)
            self.seq = self.search(problem)
            if not self.seq:
                return None
        return self.seq.pop(0)

    def update_state(self, state, percept):
        loc_row, loc_col = self.location[0], self.location[1]

        ## Note: up means increading the array rows
        #if percept == 'suck': self.perforance += 10
        #if percept == 'up': self.perforance -= 1
        #if percept == 'down': self.perforance -= 1
        #if percept == 'left': self.perforance -= 1
        #if percept == 'right': self.perforance -= 1
            
        self.location = (loc_row, loc_col)

    def formulate_goal(self, state):
        return np.array([0 for i in range(0, dim*dim)]).reshape(dim, dim)

    def formulate_problem(self, state, goal):
        return P(initial=(tuple(state.flatten()), tuple(self.location)), goal=goal)

    def search(self, problem):
        #self.solution = depth_first_graph_search(problem)
        self.solution = best_first_graph_search(problem, lambda node: sum(node.state[0]))   # node.path_cost  sum(node.state[0])   self.status.sum()
        seq = get_seq(self.solution)
        print('  > Dude, this is my search/planning result: ', seq)
        print('  >')
        return seq

In [11]:
dim = 3
env = VacuumEnv(dim, p_dirt=0.2)

# Create the search agent
search_agent = SimpleProblemSolvingAgentProgram(initial_state=env.status, location=[3,3])
env.add_thing(search_agent)
print('Initial env status:\n', env.status)
print('\nLet the cleaner loose:')
env.run(steps=100)


print('\nFinal env status:\n', env.status)

for agent in env.agents:
    print('\nAgent performance:', agent.performance)
    print('Agent path cost  :', agent.program.solution.path_cost)

Initial env status:
 [[0 0 0]
 [0 0 0]
 [1 1 0]]

Let the cleaner loose:
  > Dude, this is my search/planning result:  ['up', 'up', 'left', 'left', 'down', 'down', 'suck', 'right', 'suck']
  >
  > up in [3,3]
  > up in [2,3]
  > left in [1,3]
  > left in [1,2]
  > down in [1,1]
  > down in [2,1]
  > suck in [3,1]
  > right in [3,1]
  > suck in [3,2]
Done

Final env status:
 [[0 0 0]
 [0 0 0]
 [0 0 0]]

Agent performance: 13
Agent path cost  : 9


---

**4. Compare your best search agent with a simple randomized reflex agent that sucks if there is dirt and otherwise moves randomly.**

**5. Consider what would happen if the world were enlarged to NxN.**

In [12]:
dim = 5
env = VacuumEnv(dim, p_dirt=0.5)

# Create the search agent
search_agent = SimpleProblemSolvingAgentProgram(initial_state=env.status, location=[2,2])
env.add_thing(search_agent)
print('Initial env status:\n', env.status)
print('\nLet the cleaner loose:')
env.run()


print('\nFinal env status:\n', env.status)

for agent in env.agents:
    print('\nAgent performance:', agent.performance)
    print('Agent path cost  :', agent.program.solution.path_cost)

Initial env status:
 [[0 1 1 0 1]
 [1 1 1 0 1]
 [1 0 1 1 1]
 [0 1 1 1 0]
 [1 0 1 1 1]]

Let the cleaner loose:
  > Dude, this is my search/planning result:  ['suck', 'up', 'suck', 'right', 'suck', 'right', 'right', 'suck', 'left', 'left', 'left', 'left', 'down', 'suck', 'up', 'right', 'right', 'down', 'suck', 'up', 'right', 'right', 'down', 'suck', 'up', 'left', 'left', 'left', 'left', 'down', 'down', 'suck', 'up', 'up', 'right', 'right', 'down', 'down', 'suck', 'right', 'suck', 'right', 'suck', 'up', 'up', 'left', 'left', 'left', 'down', 'down', 'down', 'suck', 'right', 'suck', 'right', 'suck', 'up', 'up', 'up', 'left', 'left', 'left', 'down', 'down', 'down', 'down', 'suck', 'up', 'up', 'up', 'up', 'right', 'right', 'down', 'down', 'down', 'down', 'suck', 'right', 'suck', 'right', 'suck']
  >
  > suck in [2,2]
  > up in [2,2]
  > suck in [1,2]
  > right in [1,2]
  > suck in [1,3]
  > right in [1,3]
  > right in [1,4]
  > suck in [1,5]
  > left in [1,5]
  > left in [1,4]
  > left in [1

<br>

While **breadth first graph search** is capable of finding overall shorter solution paths to clean the environment, it takes exponentially more time and space in slighly increased environments, e.g. (4x4, 5x5, etc.).

**depths first graph search** to find a solution path in acceptable time and then take a longer router to clean the environment.

**best first search** finds shorter paths then **depths first graph search**, but also visits several locations multiple times.

<br>

**TODO:**
Check how search state can be modified to avoid visiting same locations.